# Reflection Technique with LangGraph

In this notebook, we'll explore the reflection technique using LangGraph. We'll create a system where one agent reflects on a tweet and provides feedback, while another agent generates an improved tweet based on that reflection. We'll use the Gemma 2 open weights model for our language tasks.

## Installation

First, let's install the necessary packages: langchain, langchain-ollama, and langgraph.

In [ ]:
!pip install langchain langchain-ollama langgraph

## Setup and Imports

Now, let's import the necessary libraries and set up our environment.

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_ollama import ChatOllama
import operator
from typing import TypedDict, Annotated, Dict, Any
from dotenv import load_dotenv
from langgraph.graph import END, StateGraph

load_dotenv()

## Defining Prompts and Chains

Now, let's define our prompts for reflection and generation, and create our language model chains.

In [ ]:
reflection_prompt = ChatPromptTemplate.from_messages([
    ("system", """
    You are a viral twitter influencer grading a tweet. Generate critique and recommendations for the user's tweet.
    Always provide detailed recommendations, including requests for length, virality, style, etc.,
    "Always return your answer in markdown format

    ```markdown
    **Original Tweet:** Just finished my first day back at work after vacation. Feeling kinda blah
    
    Answer:
    **Grade:** 5/10

    **Critique:** This tweet is a bit too generic. It lacks a hook to grab attention and doesn't really give us any insight
    into your vacation or how you're feeling.

    **Recommendations:**
    - **Length:** Keep it under 280 characters for maximum impact.
    - **Virality:** Try adding a question or call to action to encourage engagement. For example, "What's the best thing
    you did on your last vacation?"
    - **Style:** Use more descriptive language to paint a picture for your followers. Instead of 'kinda blah,' try
    something like 'overwhelmed by the return to reality' or 'missing the sunshine and cocktails.'

    **Original Tweet:**: {tweet}
    """)
])

generation_prompt = ChatPromptTemplate.from_messages([
    ("system", """
    You are a twitter techie influencer assistant tasked with revising twitter posts according
    to feedback from the previous message.
    Revise the current tweet according to the critique to the best twitter post possible.
    {critique}
    You ONLY should respond with a revised version of the tweet.
    **current tweet:** {tweet}
    There is no need for markdown format, only write the tweet and do not write the critique
    **New Tweet:** 
    """)
])

llm = ChatOllama(model="gemma2")
generate_chain = generation_prompt | llm
reflect_chain = reflection_prompt | llm

## Defining the State and Node Functions

Let's define our State type and the functions for our reflection and generation nodes.

In [ ]:
class State(TypedDict):
    tweet: str
    original_tweet: str
    critique: str
    revised_tweet: str
    iterations: int

REFLECT = "reflect"
GENERATE = "generate"

def generation_node(state: State) -> Dict[str, Any]:
    res = generate_chain.invoke({"tweet": state["tweet"], "critique": state["critique"]})
    return {"revised_tweet": res.content, "iterations": int(state["iterations"]) + 1}

def reflection_node(state: State) -> Dict[str, Any]:
    res = reflect_chain.invoke({"tweet": state["tweet"]})
    return {"critique": res.content}

## Building the Graph

Now, let's build our LangGraph using the StateGraph builder.

In [ ]:
builder = StateGraph(State)
builder.add_node(REFLECT, reflection_node)
builder.set_entry_point(REFLECT)

builder.add_node(GENERATE, generation_node)

def should_continue(state: Dict[str, Any]):
    if int(state["iterations"]) >= 2:
        return END
    return GENERATE

builder.add_conditional_edges(REFLECT, should_continue)
builder.add_edge(GENERATE, REFLECT)

graph = builder.compile()

## Visualizing the Graph

Let's visualize our graph structure.

In [ ]:
graph.get_graph().draw_mermaid_png(output_file_path="graph.png")

## Running the Graph

Finally, let's run our graph with an initial tweet and observe the reflection and generation process.

In [ ]:
if __name__ == "__main__":
    print("Hello LangGraph")
    tweet = """
    @LangChainAI
    — newly Tool Calling feature is underrated.
    After a long wait, it's  here- making the implementation of agents across different models
    with function calling - super easy.
    """

    for state in graph.stream(
        {"tweet": tweet, "revised_tweet": tweet, "iterations": 0},
        {"configurable": {"thread_id": "odsc_userid_12844"}},
        stream_mode="values",
    ):
        print(f"{state=}")

## Conclusion

In this notebook, we've implemented a reflection technique using LangGraph. We created a loop where one agent reflects on a tweet and provides feedback, while another agent generates an improved tweet based on that reflection. The process continues for a set number of iterations, demonstrating how we can manage state and create complex workflows with LangGraph.

Key points to note:
1. We use two different prompts: one for reflection and one for generation.
2. The state management allows us to keep track of the original tweet, critiques, revised tweets, and the number of iterations.
3. The conditional edge allows us to control the flow of the graph based on the number of iterations.
4. We use the Gemma 2 open weights model, showcasing the flexibility of LangGraph in working with different language models.

This technique can be extended to various applications where iterative improvement and reflection are valuable, such as content creation, code review, or any task that benefits from multiple rounds of feedback and refinement.